In [1]:
import pandas as pd

df = pd.read_csv("data/balanced_output.csv")
print(df.head())

                                                Text  Label         ID
0  ADVERTISEMENT\n\nIt's not just Osama bin Laden...  Human  D1-178627
1  Namaste Solar Inc.’s Development Strategy Case...  Human   D1-62612
2  Revolutionary Movements and Wars of Rebellion ...  Human   D1-47706
3  Harnett County Public Health IT Department Ess...  Human   D1-68331
4  More innovative gear from Bastl Instruments.\n...  Human  D1-150166


In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df["Text"])  
sequences = tokenizer.texts_to_sequences(df["Text"])
padded_sequences = pad_sequences(sequences, padding="post")

df["Label"] = df["Label"].map({"Human": 0, "AI": 1})  # Convertendo para números

2025-03-18 20:55:26.379513: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-18 20:55:26.389016: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-18 20:55:26.455395: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-18 20:55:26.499158: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742331326.550531  124621 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742331326.56

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df["Label"], test_size=0.3, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout

model_dnn = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=X_train.shape[1]),
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(64, activation="relu"),
    Dense(1, activation="sigmoid")
])

model_dnn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model_dnn.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=32)


/home/luis/miniconda3/envs/ap/lib/python3.9/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-03-18 20:55:30.788552: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 67s 2s/step - accuracy: 0.3352 - loss: nan - val_accuracy: 0.3354 - val_loss: nan
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.3168 - loss: nan - val_accuracy: 0.3354 - val_loss: nan
Epoch 3/100
11/37 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.3031 - loss: nan

In [ ]:
from tensorflow.keras.layers import SimpleRNN

model_rnn = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=X_train.shape[1]),
    SimpleRNN(64, return_sequences=True),
    SimpleRNN(32),
    Dense(1, activation="sigmoid")
])

model_rnn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model_rnn.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=32)


In [ ]:
from tensorflow.keras.layers import LSTM

model_lstm = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=X_train.shape[1]),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dense(1, activation="sigmoid")
])

model_lstm.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model_lstm.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)


In [ ]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_bert = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Pré-processar o texto
def encode_texts(texts):
    return tokenizer(texts, truncation=True, padding=True, max_length=512, return_tensors="tf")

train_encodings = encode_texts(df["Text"].tolist())
train_labels = df["Label"].values

# Treinar o modelo
model_bert.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model_bert.fit(train_encodings["input_ids"], train_labels, epochs=3, batch_size=8)


In [ ]:
from sklearn.metrics import classification_report

y_pred_dnn = (model_dnn.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred_dnn))

y_pred_rnn = (model_rnn.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred_rnn))

y_pred_lstm = (model_lstm.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred_lstm))